In [61]:
import pandas as pd
import numpy as np
import unicodedata, re
from sklearn.preprocessing import LabelEncoder

In [62]:
def load_data(name):
    tmp = pd.read_csv(f"../data/{name}")
    tmp.rename(columns={'Name': 'name',
                        'Price': 'price',
                        'Brand': 'brand',
                        'Category': 'category'}, inplace=True)
    tmp = tmp[['name', 'price', 'category', 'brand']]
    tmp.dropna(inplace=True)

    return tmp

In [63]:
files = ["df_nondim.csv", "dts.csv", "tupan_0x0.csv",
         "tupan_0x1.csv", "tupan_0x2.csv", "tupan_0x3.csv"]
dt = pd.DataFrame(columns=['name', 'price', 'category', 'brand'])
for file in files:
    aux = load_data(file)
    dt = pd.concat([dt.astype(aux.dtypes), aux.astype(dt.dtypes)],
                   ignore_index=True)
dt.drop_duplicates(subset=["name"], inplace=True)
raw_df = dt.copy(deep=True)
print(f"shape: {dt.shape}")

shape: (13060, 4)


In [64]:
stopwords = ["m", "mm", "mmx", "xmm", "mxm", "xcm", "x", "cm", "l", "ml", "x", "xm", "kg"]
stopword_regex = [r'\b' + re.escape(word) + r'\b' for word in stopwords]
def normalize_text(text):
    return unicodedata.normalize('NFD', str(text))
def filter_unicode_chars(text):
    return ''.join(char for char in text if not unicodedata.combining(char))

def remove_stopwords(text):
    for regex in stopword_regex:
        text = re.sub(regex, '', text, flags=re.IGNORECASE)
    return text

def clean_name(row):
    name = normalize_text(row).lower()
    name = re.sub(r'\s*/?\s*ref(?:\.|\s*:\s*)(\s*\w+)?', '', name, flags=re.IGNORECASE)
    name = re.sub(r'[0-9]', '', name)
    name = re.sub(r'[><\\/.,:)(°-]', '', name)
    name = re.sub(r'[+/%-]', ' ', name)
    name = remove_stopwords(name)
    name = filter_unicode_chars(name)
    name = " ".join(name.split())
    return name

def clean_category(row):
    category = normalize_text(row).lower()
    category = re.sub(r'[0-9]', '', category)
    category = filter_unicode_chars(category)
    words = category.split()
    unique_words = list(dict.fromkeys(words))
    cleaned_category = " ".join(unique_words)

    return cleaned_category

df = dt.copy()
df["category"] = df["category"].apply(clean_category)
df.head(5)

,name,price,category,brand
0,Telha Ecológica Clássica Fit 200x75cm Vermelho...,85.9,telhas > de fibra vegetal,ONDULINE
1,Pneu 325 Aro 8 com 2 Lonas Leve COLSON / REF. ...,32.9,maquinas para construcao > carro de mao,COLSON
2,Caixa para Massa de Plástico 20 Litros Reforça...,24.9,acessorios para construcao > de uso geral,DIMAX BR
3,"Esquadro em Aço 14 Polegadas x35,5cm com Cabo ...",18.9,acessorios para construcao > de uso geral,DIMAX BR
4,"Esquadro em Aço 12 Polegadas x30,4cm com Cabo ...",17.9,acessorios para construcao > de uso geral,DIMAX BR


In [65]:
values_byprefix = {"acabamentos":"acabamentos",  "acessorios agricolas":"acess_agricolas", "acessorios automotivos":"acess_automotivos",
              "acessorios de iluminacao de uso geral":"acess_iluminacao",
              ("acessorios de jardinagem", "jardim", "maquinas e ferramentas de jardinagem"):"acess_jardinagem",
              ("acessorios de tecnologia","seguranca"):"acess_tecnologia",
              ("acessorios e conexoes eletricas", "fitas e mangueiras de led", "kits trilho"):"acess_conexoes_eletricas",
              "acessorios e utensilios de cozinha":"acess_utensilios_cozinha", "acessorios e utensilios de lavanderia":"acess_utensilios_lavanderia",
              ("acessorios para construcao", "ferramentas para construcao", "maquinas para construcao"):"acess_construcao",
              "acessorios para decoracao":"acess_decoracao", "acessorios para ferramentas":"acess_ferramentas", "acessorios para pintura":"acess_pintura",
              "acessorios para pisos e revestimentos":"acess_pisos_revestimentos", "acessorios sanitarios":"acess_sanitarios", "aco":"aco",
              ("adornos", "outros adornos"):"adornos", "arandelas":"arandelas",
              ("argamassas e rejuntes", "massa", "materiais de campo"):"argamassas_rejuntes",
              "assentos sanitarios":"assentos_sanitarios", "bacias e caixas":"bacias_caixas", "banho":"banho", "bombas e filtros":"bombas_filtros",
              "brocas":"brocas", "cabos, fios e conectores":"cabos_fios_conectores", "caixas, quadros e disjuntores":"caixas_quadros_disjuntores",
              "cama":"cama", "casa":"casa", "churrasco":"churrasco", "chuveiros":"chuveiros", "complementos":"complementos", "conexoes":"conexoes",
              "cortinas e persianas":"cortinas_persianas", "cozinha":"cozinha", "cubas, pias e balcoes":"cubas_pias_balcoes",
              "discos e rebolos":"discos_rebolos", "duchas higienicas":"duchas_higienicas", "eletroportateis":"eletroportateis", "epi":"epi",
              "fechaduras":"fechaduras",
              ("ferramentas a bateria", "ferramentas eletricas"):"ferramentas_eletricas", "ferramentas agricolas":"ferramentas agricolas",
              "ferramentas manuais":"ferramentas_manuais",
              "impermeabilizantes":"impermeabilizantes", ("interruptores", "tomadas"):"interruptores_tomadas",
              "janelas":"janelas", "lampadas":"lampadas", "lavatorios":"lavatorios", "limpeza":"limpeza", "lonas":"lonas",
              ("luminarias", "refletores", "spots"):"luminarias_refletores",
              "moveis":"moveis", "paineis de led":"paineis_led", "pinceis":"pinceis", "pisos":"pisos", "porcelanatos":"porcelanatos", "portas":"portas",
              "prateleiras":"prateleiras", "registros":"registros", "revestimentos":"revestimentos", "tintas":"tintas", "torneiras":"torneiras",
              ("piscinas","tratamento"):"piscinas_tratamento", "tubos":"tubos", "ferragens":"ferragens"
              }
values_bysufix = {("grelhas", "calhas"): "acess_hidraulico", "acessorios hidraulicos de uso geral": "acess_hidraul_geral"}

In [66]:
def replace_by_prefix(row):
    for key, value in values_byprefix.items():
        if row.startswith(key):
            return value
    return row
def replace_by_sufix(row):
    for key, value in values_bysufix.items():
        if row.endswith(key):
            return value
    return row

df["category"] = df.category.apply(replace_by_prefix)
df["category"] = df.category.apply(replace_by_sufix)
df.shape

(13060, 4)

In [67]:
df = df[df["category"].isin(values_byprefix.values()) | df["category"].isin(values_bysufix.values())]
df

,name,price,category,brand
1,Pneu 325 Aro 8 com 2 Lonas Leve COLSON / REF. ...,32.9,acess_construcao,COLSON
2,Caixa para Massa de Plástico 20 Litros Reforça...,24.9,acess_construcao,DIMAX BR
3,"Esquadro em Aço 14 Polegadas x35,5cm com Cabo ...",18.9,acess_construcao,DIMAX BR
4,"Esquadro em Aço 12 Polegadas x30,4cm com Cabo ...",17.9,acess_construcao,DIMAX BR
6,Cimento Forte Multiuso CPII 50kg CIMENTO FORTE...,34.89,argamassas_rejuntes,CIMENTO FORTE
...,...,...,...,...
54310,Adaptador Soldável PVC 40x1.1/4 Curto - Ref.22...,4.49,conexoes,TIGRE
54311,Adaptador Soldável Curto PVC 85X3mm - Ref. 220...,31.9,conexoes,TIGRE
54312,Adaptador Soldável PVC 50x1.1/2 Curto - Ref.22...,4.19,conexoes,TIGRE
54313,Adaptador Soldável PVC 20x1/2 Curto - Ref.2219...,0.99,conexoes,TIGRE


In [68]:
df["name"] = df["name"].apply(clean_name)
df.drop_duplicates(subset=["name"], inplace=True)
df

,name,price,category,brand
1,pneu aro com lonas leve colson,32.9,acess_construcao,COLSON
2,caixa para massa de plastico litros reforcada ...,24.9,acess_construcao,DIMAX BR
3,esquadro em aco polegadas com cabo de plastico...,18.9,acess_construcao,DIMAX BR
6,cimento forte multiuso cpii cimento forte,34.89,argamassas_rejuntes,CIMENTO FORTE
7,linha polietileno mmxm para pedreiro lisa nove,5.9,acess_construcao,NOVE54
...,...,...,...,...
54300,arco serra fixo em aco polegadas com cabo poli...,35.9,ferramentas_manuais,TRAMONTINA
54301,arco de serra em aco polegadas fixo eco com ca...,25.9,ferramentas_manuais,DIMAX
54305,mini arco de serra polegadas com corpo injetad...,22.9,acess_ferramentas,TRAMONTINA
54306,adaptador soldavel pvc curto tigre,2.29,conexoes,TIGRE


In [93]:
categories = df.category.value_counts()
first_seg_cat = categories[categories >= 200].index
second_seg_cat = categories[(categories < 200) & (categories > 70)].index
df_fs = df[df.category.isin(first_seg_cat)]
df_sn = df[df.category.isin(second_seg_cat)]

In [79]:
df_fs.category.value_counts()

category
tintas                   547
cozinha                  435
torneiras                422
acess_sanitarios         373
interruptores_tomadas    366
ferragens                341
fechaduras               334
complementos             326
acess_ferramentas        246
conexoes                 215
pisos                    211
porcelanatos             204
Name: count, dtype: int64

In [71]:
df_sn.category.value_counts()

category
casa                          98
impermeabilizantes            89
assentos_sanitarios           88
ferramentas_eletricas         87
acess_construcao              84
caixas_quadros_disjuntores    80
brocas                        78
bacias_caixas                 71
paineis_led                   71
cabos_fios_conectores         68
acabamentos                   66
acess_decoracao               65
portas                        65
acess_pisos_revestimentos     60
eletroportateis               59
duchas_higienicas             51
arandelas                     48
lavatorios                    47
acess_tecnologia              47
cortinas_persianas            46
acess_utensilios_cozinha      35
tubos                         35
janelas                       34
ferramentas agricolas         32
adornos                       30
bombas_filtros                28
piscinas_tratamento           24
acess_agricolas               22
churrasco                     20
lonas                         13
a

In [81]:
lab = LabelEncoder()
df_fs.loc[:, "target"] = lab.fit_transform(df_fs["category"])
df_sn.loc[:, "target"] = lab.fit_transform(df_sn["category"])

In [84]:
df_fs.to_csv("../data/df_fs.csv", index=False)
df.to_csv("../data/clean_df.csv", index=False)
dt.to_csv("../data/full_df.csv", index=False)

##Melhores hiperparâmetros encontrados: AdamW 0.7850667834281921
 - dense1_units: 320
 - dense2_units: 96
 - dense3_units: 160
 - dense_layer_final: 64
 - dropout1: 0.4
 - dropout2: 0.4
 - dropout3: 0.4
 - l2_reg: 0.0027560779294081086
 - alpha: 0.2
 - learning_rate: 0.0006263130573774603

## Melhores hiperparâmetros encontrados: Adam
Epoch 20: val_loss improved from 0.75603 to 0.75577, saving model to best_bert_siamese_model.keras

64/64 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - loss: 0.6773 - val_loss: 0.7558
 - dense1_units: 448
 - dense2_units: 128
 - dense3_units: 128
 - dense_layer_final: 128
 - dropout1: 0.30000000000000004
 - dropout2: 0.4
 - dropout3: 0.4
 - l2_reg: 0.001615253021487585
 - alpha: 0.2
 - learning_rate: 0.0036321404388540523
#=====================================

Best val_loss So Far: 1.1385364532470703
Total elapsed time: 00h 56m 15s

Search: Running Trial #33

Value             |Best Value So Far |Hyperparameter
256               |384               |dense1_units
96                |160               |dense2_units
0.4               |0.4               |dropout1
0.0027186         |0.0015619         |l2_reg
0.2               |0.2               |dropout2
0.8               |0.2               |alpha
0.0089625         |0.00060028        |learning_rate

## Best val_loss So Far: 1.3455398082733154
Melhores hiperparâmetros encontrados:
 - dense1_units: 320
 - dense2_units: 192
 - dropout1: 0.2
 - dropout2: 0.30000000000000004
 - l2_reg: 0.0011285276077827135
 - alpha: 0.2
 - learning_rate: 0.00213954964760784
## Best val_loss So Far: 1.8878151178359985
Total elapsed time: 00h 40m 59s
Melhores hiperparâmetros encontrados:
 - dense1_units: 512
 - dense2_units: 96
 - dropout1: 0.4
 - dropout2: 0.30000000000000004
 - l2_reg: 0.002288147780992941
 - alpha: 0.2
 - learning_rate: 0.0009587343827472403